In [1]:
from pathlib import Path

import pandas as pd

from pybenchmark.client import BenchmarkClient

In [2]:
URL = "http://127.0.0.1:8000"
DATA_DIR = Path("../resources")

In [3]:
client = BenchmarkClient(url=URL)

# Muscle Groups

In [4]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="musclegroups")
df.head()

,name
0,chest
1,triceps
2,biceps
3,shoulders
4,quadriceps


In [5]:
for name in df.name.tolist():
    try:
        response = client.musclegroups.create_musclegroup(name=name)
        print(f"Created Muscle Group {name}")
    except Exception as e:
        print(e)

Created Muscle Group chest
Created Muscle Group triceps
Created Muscle Group biceps
Created Muscle Group shoulders
Created Muscle Group quadriceps
Created Muscle Group hamstrings
Created Muscle Group calves
Created Muscle Group core
Created Muscle Group back
Created Muscle Group glutes


# Exercises

In [6]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="exercises")
df.head()

,name,musclegroups
0,Bench Press,"chest, triceps"
1,Lean-In Cable Row,"back, biceps"
2,Dumbbell Shoulder Press Neutral Grip,shoulders
3,Lat Pulldown Neutral Grip,"back, biceps"
4,Rope Hammer Curls,biceps


In [7]:
mgs = pd.DataFrame(client.musclegroups.get_musclegroups())
mgs.head()

,name,id
0,chest,1
1,triceps,2
2,biceps,3
3,shoulders,4
4,quadriceps,5


In [8]:
for ix, exercise in df.iterrows():
    name = exercise["name"]
    mg_names = [mg.strip() for mg in exercise.musclegroups.split(",")]
    mg_ids = mgs[mgs["name"].isin(mg_names)].id.to_list()
    try:
        response = client.exercises.create_exercise(name=name, musclegroup_ids=mg_ids)
        print(f"Created Exercise {name}")
    except Exception as e:
        print(e)

Created Exercise Bench Press
Created Exercise Lean-In Cable Row
Created Exercise Dumbbell Shoulder Press Neutral Grip
Created Exercise Lat Pulldown Neutral Grip
Created Exercise Rope Hammer Curls
Created Exercise Cable Lateral Raises
Created Exercise Tricep Pushdown
Created Exercise V-Bar Pushdown
Created Exercise Barbell Back Squats
Created Exercise Barbell Stiff Leg Deadlift
Created Exercise Dumbbell Bulgarian Split Squats Single Arm
Created Exercise Hyperextensions
Created Exercise Rollouts
Created Exercise Military Press
Created Exercise Pull Ups Neutral Grip
Created Exercise Incline Dumbbell Bench Press Neutral Grip
Created Exercise Dumbbell Prone Rows
Created Exercise Seated Dumbbell Hammer Curls
Created Exercise Seated Dumbbell Lat Raises
Created Exercise Dumbbell Skull Crushers
Created Exercise Trap Bar Deadlift
Created Exercise Dumbbell Split Squats
Created Exercise Lying Leg Curls
Created Exercise Leg Press
Created Exercise Leg Raises
Created Exercise Hack Squat
Created Exerc

# Workout Routines

In [9]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workoutroutines")
df = df.rename(columns={"name": "routine_name"})
df.head()

,routine_name,exercise,reps
0,Wave Loading 321 Lower 1,Trap Bar Deadlift,3
1,Wave Loading 321 Lower 1,Trap Bar Deadlift,2
2,Wave Loading 321 Lower 1,Trap Bar Deadlift,1
3,Wave Loading 321 Lower 1,Trap Bar Deadlift,3
4,Wave Loading 321 Lower 1,Trap Bar Deadlift,2


In [10]:
routines = df["routine_name"].unique()
routines

array(['Wave Loading 321 Lower 1', 'Wave Loading 321 Upper 1',
       'Wave Loading 321 Lower 2', 'Wave Loading 321 Upper 2',
       'Wave Loading 642 Lower 1', 'Wave Loading 642 Upper 1',
       'Wave Loading 642 Lower 2', 'Wave Loading 642 Upper 2',
       'Hypertrophy Legs', 'Hypertrophy Chest and Back',
       'Hypertrophy Shoulders and Arms', 'Arm Block Chest',
       'Arm Block Back', 'Arm Block Legs', 'Arm Block Arms',
       'Arm Block Legs 2'], dtype=object)

In [11]:
exs = pd.DataFrame(client.exercises.get_exercises())
exs.head()

,name,id
0,bench press,1
1,lean-in cable row,2
2,dumbbell shoulder press neutral grip,3
3,lat pulldown neutral grip,4
4,rope hammer curls,5


In [12]:
df["exercise_lower"] = df.exercise.str.lower()
df.head()

,routine_name,exercise,reps,exercise_lower
0,Wave Loading 321 Lower 1,Trap Bar Deadlift,3,trap bar deadlift
1,Wave Loading 321 Lower 1,Trap Bar Deadlift,2,trap bar deadlift
2,Wave Loading 321 Lower 1,Trap Bar Deadlift,1,trap bar deadlift
3,Wave Loading 321 Lower 1,Trap Bar Deadlift,3,trap bar deadlift
4,Wave Loading 321 Lower 1,Trap Bar Deadlift,2,trap bar deadlift


In [13]:
df_all = df.merge(right=exs, left_on="exercise_lower", right_on="name").set_index("routine_name")[["reps", "id"]]
df_all = df_all.reset_index().set_index(["routine_name", "id"])
df_all

reps
routine_name             id      
Wave Loading 321 Lower 1 21     3
                         21     2
                         21     1
                         21     3
                         21     2
...                           ...
Arm Block Legs 2         47    24
                         42    20
                         42    20
                         42    20
                         42    20

[333 rows x 1 columns]

In [14]:
for routine in routines:
    all_exercises = []
    exercises = df_all.loc[routine].index.unique()
    for exercise in exercises:
        sets = df_all.loc[routine, exercise].to_dict(orient="records")
        data = {"id": exercise, "planned_sets": sets}
        all_exercises.append(data)
    try:
        response = client.workout_routines.create_workout_routine(name=routine, exercises=all_exercises)
        print(f"Created Routine {routine}")
    except Exception as e:
        print(e)

C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

Created Routine Wave Loading 321 Lower 1


C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

Created Routine Wave Loading 321 Upper 1


C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

...

C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

Created Routine Arm Block Legs


C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

Created Routine Arm Block Arms
Created Routine Arm Block Legs 2


C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\774130799.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")


# Workouts

In [15]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workouts")
df = df.rename(columns={"name": "routine_name"})
df.head()

,workoutroutine,date,exercise,weight,reps
0,Hypertrophy Legs,2023-04-22,Lying Leg Curls,35.0,16
1,Hypertrophy Legs,2023-04-22,Lying Leg Curls,35.0,16
2,Hypertrophy Legs,2023-04-22,Lying Leg Curls,35.0,10
3,Hypertrophy Legs,2023-04-22,Hack Squat,60.0,8
4,Hypertrophy Legs,2023-04-22,Hack Squat,60.0,6


In [16]:
dates = df["date"].unique()
dates = [str(date.date()) for date in dates]

In [17]:
exs = pd.DataFrame(client.exercises.get_exercises())
exs = exs.rename(columns={"id": "exercise_id", "name": "exercise_name"})
exs.head()

,exercise_name,exercise_id
0,bench press,1
1,lean-in cable row,2
2,dumbbell shoulder press neutral grip,3
3,lat pulldown neutral grip,4
4,rope hammer curls,5


In [18]:
routines = pd.DataFrame(client.workout_routines.get_workout_routines())
routines = routines.rename(columns={"name": "routine_name", "id": "routine_id"})
routines.head()

,routine_name,routine_id,routine_exercises
0,wave loading 321 lower 1,1,"[{'exercise': {'name': 'trap bar deadlift', 'i..."
1,wave loading 321 upper 1,2,"[{'exercise': {'name': 'bench press', 'id': 1}..."
2,wave loading 321 lower 2,3,"[{'exercise': {'name': 'barbell back squats', ..."
3,wave loading 321 upper 2,4,"[{'exercise': {'name': 'military press', 'id':..."
4,wave loading 642 lower 1,5,"[{'exercise': {'name': 'trap bar deadlift', 'i..."


In [19]:
df["exercise_lower"] = df.exercise.str.lower()
df["workoutroutine_lower"] = df.workoutroutine.str.lower()
df.head()

,workoutroutine,date,exercise,weight,reps,exercise_lower,workoutroutine_lower
0,Hypertrophy Legs,2023-04-22,Lying Leg Curls,35.0,16,lying leg curls,hypertrophy legs
1,Hypertrophy Legs,2023-04-22,Lying Leg Curls,35.0,16,lying leg curls,hypertrophy legs
2,Hypertrophy Legs,2023-04-22,Lying Leg Curls,35.0,10,lying leg curls,hypertrophy legs
3,Hypertrophy Legs,2023-04-22,Hack Squat,60.0,8,hack squat,hypertrophy legs
4,Hypertrophy Legs,2023-04-22,Hack Squat,60.0,6,hack squat,hypertrophy legs


In [20]:
df_all = df.merge(right=exs, left_on="exercise_lower", right_on="exercise_name")
df_all = df_all.merge(right=routines, left_on="workoutroutine_lower", right_on="routine_name")[["date", "weight", "reps", "routine_id", "exercise_id"]]
df_all = df_all.set_index(["date", "routine_id", "exercise_id"])
df_all

weight  reps
date       routine_id exercise_id              
2023-04-22 9          23             35.0    16
                      23             35.0    16
                      23             35.0    10
                      26             60.0     8
                      26             60.0     6
...                                   ...   ...
2024-11-18 12         8              20.0    20
                      8              22.5    14
                      8              20.0    17
                      39              0.0    17
                      39              0.0    13

[296 rows x 2 columns]

In [21]:
for date in dates:
    all_workouts = []
    routines = df_all.loc[date].index.get_level_values(0).unique()
    for routine in routines:
        all_exercises = []
        exercises = df_all.loc[(date, routine)].index.unique()
        for exercise in exercises:
            sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
            data = {"id": exercise, "sets": sets}
            all_exercises.append(data)
        try:
            response = client.workouts.create_workout(date=str(date), exercises=all_exercises, workoutroutine_id=routine)
            print(f"Created Workout for date {date}")
        except Exception as e:
            print(e)

C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercises = df_all.loc[(date, routine)].index.unique()
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, ro

Created Workout for date 2023-04-22
Created Workout for date 2023-04-21


C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercises = df_all.loc[(date, routine)].index.unique()
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, ro

Created Workout for date 2024-10-28


C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercises = df_all.loc[(date, routine)].index.unique()
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, ro

...

Created Workout for date 2024-11-13
Created Workout for date 2024-11-14


C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercises = df_all.loc[(date, routine)].index.unique()
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, ro

Created Workout for date 2024-11-15


C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercises = df_all.loc[(date, routine)].index.unique()
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_22804\2834727724.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, ro

Created Workout for date 2024-11-18
